# Model 1: Tipus incidència 'No ha impartit classe a aquest grup'
## Part-of-speech method

En este notebook se va ha intendar mejorar el modelo de detección de la incidencia 
'No ha impartit classe a aquest grup' mediante el uso de Part-of-Speech y Dependency Parsing.

Aquí se evaluarán las reglas definidas con los datos de muestra.

In [1]:
#import json
#import os.path
#import sys
import csv
import pandas as pd
#import numpy as np
#from colorama import Fore

In [2]:
#from __future__ import unicode_literals, print_function
#import plac
#import random
#from pathlib import Path

In [3]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher

In [4]:
def load_data(language):
    
    if (debug >= 1):
        print ("LOAD_DATA")
        
    # Load data from file
    file = "comentaris_" + language + ".csv"
    data = pd.read_csv(pathdest + file)
    if (debug >= 1):
        print ("Original data:", data.shape[0])
    if (debug >= 2):
        display (data.sample(5))
        
    # Calculates label and filter rows to be used    
    data_prof = data[data.TipusPregunta == "P"][["Comentari","TipusIncidencia"]]
    if (debug >= 1):
        print("Filtered data: ", data_prof.shape[0])        
    if (debug >= 2):
        display (data_prof.sample(5))
        
    # Calculates labels
    data_prof["label"] = data_prof["TipusIncidencia"] == "No ha impartit classe a aquest grup"    
    data_fin = data_prof[["Comentari","label"]]
    
    if (debug >= 1):
        print ("Final data:", data_fin.shape[0])
    if (debug >= 2):
        display (data_fin.sample(5))
        
    return data_fin

In [69]:
def train_matcher(nlp):
    
    if (debug >= 1):
        print ("TRAIN MATCHER")
  
    # Rule-based matching
    matcher = Matcher(nlp.vocab, validate=True)

    adv_negs = ["no","tampoco"]
    verbs = ["tratar","ir","conocer","ver","saber","ser","aparecer","tener"]
    verbs_trans = ["realizar","dar","recibir","dictar"]
    noms_trans = ["clase","asignatura"]

    # Example: "No he tratado con é", "No traté", "No he recibido", "No recibí"
    pattern = [{"POS": "ADV","LOWER":{"IN":adv_negs}}, {"OP": "*"}, 
               {"DEP":"ROOT","POS": "VERB", "LEMMA": {"IN": verbs}}]
    matcher.add("NoClasses1", None, pattern)

    # Example: "No ha realizado clases"
    pattern = [{"POS": "ADV","LOWER":{"IN":adv_negs}}, {"OP": "*"}, 
               {"DEP":"ROOT","POS": "VERB", "LEMMA": {"IN": verbs_trans}},
               {"OP": "*"}, {"POS":"NOUN","LEMMA":{"IN":noms_trans}}]
    matcher.add("NoClasses3", None, pattern)

    if (debug >= 1):
        print ("Patterns: ", len(matcher))
    
    return matcher

In [51]:
def test_matching(nlp, matcher, text):
    
    predict = False;
    if (debug >= 2):
        print ("PREDICT TEXT:")
    
    doc = nlp(text)
    for sent in doc.sents:
        if (debug >= 2):
            print (sent.text)
            
        matches = matcher(sent)
        if (len(matches) == 0):
            if (debug >= 2):
                print ("No matches")
#            return False
        else:
            match_id, start, end = matches[len(matches)-1]
            string_id = nlp.vocab.strings[match_id] # Get string representation
            span = sent[start:end] # The matched span
            if (debug >= 2):
                print (match_id, string_id, start, end, span.text)
            predict = True
        
    return predict

In [31]:
def evaluate(nlp, matcher, data):
    fp_list = list()
    fn_list = list()

    if (debug == 1):
        print ("EVALUATE")

    tp = 0.0  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 0.0  # True negatives

    for index, row in data.iterrows():
        text = row["Comentari"]
        label = row["label"]

        result = test_matching (nlp, matcher, text)

        if (label == True):
            if (result == True):
                tp += 1.0
            else:
                fn += 1.0
                fn_list.append(text)
            if (debug >= 2):
                print ("fn: ", text)    
        else:
            if (result == True):
                fp += 1.0
                fp_list.append(text)
                if (debug >= 2):
                    print ("fp: ", text)              
            else:
                tn += 1.0

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    if (precision + recall) == 0:
        f_score = 0.0
    else:
        f_score = 2 * (precision * recall) / (precision + recall)

    return {"textcat_p": precision, "textcat_r": recall, "textcat_f": f_score, 
            "false_positive": fp_list, "false_negative": fn_list}
    
    
    
#    print (text, label, result)

In [59]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
pathmodel = "../data/models/"
debug = 1

In [49]:
languages = {"ca":"ca_fasttext_wiki", "es":"es_core_news_sm", "en":"en_core_web_sm" }
#languages = {"es":"es_core_news_lg"}

language = "es"
model = languages[language]

# Load the model form spacy
nlp = spacy.load(model)

data = load_data(language)

LOAD_DATA
Original data: 1761
Filtered data:  942
Final data: 942


In [70]:

matcher = train_matcher(nlp)
scores = evaluate(nlp, matcher, data)

print("Precission (p): ", scores["textcat_p"])
print("Recall (r): ", scores["textcat_r"])
print("F-score (f): ", scores["textcat_f"])
print("False positives: ", len(scores["false_positive"]))
display(scores["false_positive"][0:5])
print("False negatives: ", len(scores["false_negative"]))
display(scores["false_negative"][0:5])


TRAIN MATCHER
Patterns:  2
EVALUATE
Precission (p):  0.37499999995312494
Recall (r):  0.7894736840027701
F-score (f):  0.5084745761850042
False positives:  50


['No vimos a en clase a este docente ',
 'No lo sé porque no dictó clases. Debería controlar que el docente que sí lo hizo cumpla con el horario',
 'facilita bastantes conocimientos sobre los aspectos que trata. desde el principio no se sabe qué se va a ensenar y por qué. la evaluacion es complicada. dificil saber qué espera exactamente y qué ha ha sido bueno o malo despues de la evaluacion. ',
 'Joan Lluis no tiene una formación docente y eso se nota. Es accesible y amable con los estudiantes pero sus métodos no son claros. Las consignas de los trabajos no tienen concordancia con lo visto en clase y no proporciona bibliografía más que el power point de clase, que es armado por él. Esto hacer que la información vertida allí sea poco contrastable y hasta opinable. Sin embargo, creo que las clases han sido interesantes, he aprendido cosas nuevas. ',
 'Creo que no se adapta para nada a la situación que estamos viviendo, he visto una clara diferencia entre esta asignaturas y las demás resp

False negatives:  8


['El aspecto positivo es su personalidad y el interés porque aprendamos.\nEn negativo, esperaba que nos hubiese dado ella una clase para poder valorarla. ',
 'No fue nunca a clases',
 'La clase no la dictó ella. ',
 'No se quien es',
 'no se qui es aquet']